In [1]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import fisher_exact

In [2]:
df = pd.read_csv('../nyc_hiv/MPXV_HIV_matching.csv')
df.columns = ['HIV Outcomes', 'Total N', 'Outcome n', 'MPXV clustered, HIV+',
       'MPXV clustered, HIV-', 'Not MPXV clustered, HIV+', 'Not MPXV clustered, HIV-']
# convert first column to index
df = df.set_index('HIV Outcomes')
# convert all columns to float
df = df.astype(int)


def odds_ratio(a, b, c, d):
    if a == 0 or b == 0 or c == 0 or d == 0:
        return np.nan
    OR = (a/b) / (c/d)
    return OR


def odds_confidence_interval(a, b, c, d, alpha=0.05):
    if a == 0 or b == 0 or c == 0 or d == 0:
        return np.nan
    ln_or = np.log(odds_ratio(a, b, c, d))
    se_ln_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    z = norm.ppf(1-alpha/2)
    # convert to string with 1 decimal place each
    ci = np.exp(ln_or - z*se_ln_or), np.exp(ln_or + z*se_ln_or)
    ci = tuple(["{:.1f}".format(x) for x in ci])
    return ci


def fit_logistic_regression(a, b, c, d):
    # Prepare the dataset
    # 'Outcome' is 1 for positive and 0 for negative
    # 'Condition' is 1 for Condition+ and 0 for Condition-
    outcomes = [1]*a + [0]*b + [1]*c + [0]*d
    condition = [1]*a + [1]*b + [0]*c + [0]*d

    # Create DataFrame
    data = pd.DataFrame({
        'Outcome': outcomes,
        'Condition': condition
    })

    # Fit the logistic regression model
    data['Intercept'] = 1  # add an intercept
    logit_model = sm.Logit(data['Outcome'], data[['Intercept', 'Condition']])
    result = logit_model.fit(disp=0)  # suppress fitting output

    # Print the results
    # print(result.summary())

    # Odds ratio for 'Condition'
    odds_ratio = np.exp(result.params['Condition'])
    # Confidence Interval
    conf_int = np.exp(result.conf_int().loc['Condition'])

    return odds_ratio, conf_int[0], conf_int[1], result.pvalues['Condition']


def exact_fisher(a, b, c, d):
    fisher = fisher_exact([[a, b], [c, d]])
    return 'N/A', 'N/A', 'N/A', fisher[1]


def format_values(OR, CI_lower, CI_upper, p_value):
    if OR == 'N/A':
        return 'N/A', 'N/A', "{:.1e}".format(p_value)
    else:
        return "{:.2f}".format(OR), f"{CI_lower:.2f}" + '-' + f"{CI_upper:.2f}", "{:.2e}".format(p_value)


def run_analysis(a, b, c, d):
    a = int(a); b = int(b); c = int(c); d = int(d)
    if a < 5 or b < 5 or c < 5 or d < 5:
        return format_values(*exact_fisher(a, b, c, d))
    else:
        return format_values(*fit_logistic_regression(a, b, c, d))



df['OR'], df['OR_CI'], df['p-value'] = zip(*df.apply(lambda x: run_analysis(x['MPXV clustered, HIV+'], x['MPXV clustered, HIV-'], x['Not MPXV clustered, HIV+'], x['Not MPXV clustered, HIV-']), axis=1))

df

,Total N,Outcome n,"MPXV clustered, HIV+","MPXV clustered, HIV-","Not MPXV clustered, HIV+","Not MPXV clustered, HIV-",OR,OR_CI,p-value
HIV Outcomes,,,,,,,,,
HIV Diagnosis,756,328,271,321,57,107,1.58,1.11-2.27,1.21e-02
Reported HIV Genotype,328,187,155,116,32,25,1.04,0.59-1.86,8.84e-01
HIV Clustered,187,96,81,74,15,17,1.24,0.58-2.66,5.80e-01
HIV Cluster Growth,187,32,31,124,1,31,N/A,N/A,1.9e-02
HIV Recent Link,187,15,15,140,0,32,N/A,N/A,7.8e-02
